In [ ]:
# imports
import sys
sys.path.insert(1, 'modules')

In [ ]:
# local imports
import config as cfg
import protein as prot

In [ ]:
# PRE-PROCESSING TRAINING SET
# PARSING FASTA
toxic_proteins = hf.parse_fasta(cfg.f_train_toxic_fasta, 1)
atoxic_proteins = hf.parse_fasta(cfg.f_train_atoxic_fasta, 0)

In [ ]:
# print test
print(f'Total toxic sequences: {len(toxic_proteins)}')
print(f'Total atoxic sequences: {len(atoxic_proteins)}')